In [1]:
%load_ext autoreload
%autoreload 2

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
# import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
# from gensim.models import Word2Vec
# from nltk.tokenize import word_tokenize  # You may need to install nltk and download punkt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
from utilities import *
from sklearn.feature_selection import f_classif

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [2]:
# pick full or smaller version of dataset
df = pd.read_csv('data/modelready_220423.csv')

## Cleaning and baseline models

In [3]:
# print non-numerical columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
print(f'tot columns = {len(df.columns)}, numeric type columns = {len(df.select_dtypes(include=numerics).columns)}' ) # not too many non-numeric columns
df.select_dtypes(include = ['object']).head(1)  

tot columns = 772, numeric type columns = 765


,publication_number,company_name,countries_in_family,publn_nr,primary_cpc,abstract,description_text
0,US-8623043-B1,"Entellus Medical, Inc.",['AU' 'EP' 'CA' 'US'],8623043,A61M29/02,A method of treating a constricted sinus passa...,RELATED APPLICATIONS \n This Application i...


In [4]:
# extract unique countries in the df
unique_values = set()
df['countries_in_family'].apply(lambda x: unique_values.update(x.strip("[]").replace("'", "").split())) 

# Create new columns for each unique value
for value in unique_values:
    # each country has a column (1 if the patent belong to the country 0 otherwise)
    df[value] = df['countries_in_family'].apply(lambda x: 1 if value in x else 0)


In [5]:
df = df[df.abstract.notna()].copy() # drop all samples without abstract
print('missing value in description text' , df.description_text.isna().sum()) # description_text doesn't have mssing vales

missing value in description text 0


In [6]:
# encode company names
df['company_name_encoded'] = df.company_name.astype('category').cat.codes  # encode companies

# remove non-numeric columns
df_columns_dropped = df.drop(['publication_number', 'company_name', 'countries_in_family', 'publn_nr','primary_cpc'], axis = 1)

# f0_ has the same value as commercialization, the other two shouldn't be used
df_columns_dropped = df_columns_dropped.drop(['f0_', 'centrality', 'similarity'], axis = 1)

In [7]:
# remove text as I can't compute min and max on it
text = df_columns_dropped[['abstract', 'description_text']] # putting them aside for later
df_columns_dropped.drop(['abstract', 'description_text'], axis=1, inplace=True)

In [8]:
# extracting what we'll try to predict
y = df_columns_dropped['commercialized']
df_columns_dropped.drop('commercialized', axis= 1, inplace=True)

In [9]:
# dropping columns where all the value are the same (only one unique value) they would be zero if I apply min max rescaling
nunique = df_columns_dropped.nunique()
cols_to_drop = nunique[nunique == 1].index
df_clean = df_columns_dropped.drop(cols_to_drop, axis=1)

In [10]:
# splitting train and test and trying best preprocessing on training set
# all preprocessing will be done on X_train and only in the end tested on X_test
X_train, X_test, y_train, y_test = train_test_split(df_clean, y, test_size=0.20)

### Select best way to fill missin values

In [11]:
# TODO: rerun
# find best method for filling missing values

accuracies = []

#rescale 
scaler = ('scaler', StandardScaler())
filling_methods = ['mean', 'median', 'most_frequent', 'zero']

for method in filling_methods:
    # Make a copy of the original data to avoid modifying it
    data_filled = X_train.copy()

    # Fill NaN values based on the selected method
    if method == 'zero':
        # data_filled = data_filled.fillna(0)  # You can choose any constant value
        imputer =  ('imputer', SimpleImputer(strategy='constant', fill_value=0))
    else:
        imputer = ('imputer', SimpleImputer(strategy=method, keep_empty_features=False))
    
    # default max_iter was reached, so increased it
    model = ('model', LogisticRegression(max_iter=1000))

    #rescale 
    # scaler.fit(data_filled)
    # data_filled = scaler.transform(data_filled)

    pipeline = Pipeline([imputer ,scaler, model])

    # Perform cross-validation and print the mean score
    scores_dict = cross_validate(estimator=pipeline, X=data_filled, y=y_train, n_jobs=5, cv=5,scoring=['f1', 'accuracy', 'precision', 'recall'])

    accuracies.append(scores_dict["test_accuracy"])

    # Print the results
    print(f'method:{method}')
    print(f'Average F1 Score: {np.mean(scores_dict["test_f1"])}')
    print(f'Average Accuracy: {np.mean(scores_dict["test_accuracy"])}')
    print(f'Average Precision: {np.mean(scores_dict["test_precision"])}')
    print(f'Average Recall: {np.mean(scores_dict["test_recall"])}')
    print() 

method:mean
Average F1 Score: 0.8763201883062935
Average Accuracy: 0.892427433252333
Average Precision: 0.8875829759923939
Average Recall: 0.8653785071466384

method:median
Average F1 Score: 0.8764296230570316
Average Accuracy: 0.8925439888011878
Average Precision: 0.8878625594971841
Average Recall: 0.8653255690841716

method:most_frequent
Average F1 Score: 0.8767494768495926
Average Accuracy: 0.8928703780379091
Average Precision: 0.8885779562606804
Average Recall: 0.8652726310217046

method:zero
Average F1 Score: 0.8767252986105802
Average Accuracy: 0.8928470571442881
Average Precision: 0.8885315452810889
Average Recall: 0.8652726310217046



# Tests to explore the dataset

In [12]:
df

,publication_number,company_name,commercialized,vpm_patent_score,backward_citations_app,backward_citations_exa,forward_citations,total_nb_claims,nb_indep_claims,family_size,...,GR,RO,GE,CR,HK,IN,DE,HN,KR,company_name_encoded
0,US-8623043-B1,"Entellus Medical, Inc.",0,0.000000,140,18,22,11,1,18,...,0,0,0,0,0,0,0,0,0,196
1,US-9192748-B2,"Entellus Medical, Inc.",0,0.000000,203,33,2,16,2,5,...,0,0,0,0,0,0,0,0,0,196
2,US-8888686-B2,"Entellus Medical, Inc.",0,0.000000,69,8,2,15,1,2,...,0,0,0,0,0,0,0,0,0,196
3,US-8986340-B2,"Entellus Medical, Inc.",0,0.000000,162,2,3,22,1,23,...,0,0,0,0,0,0,0,0,0,196
4,US-9320876-B2,"Entellus Medical, Inc.",0,0.000000,208,2,0,10,2,18,...,0,0,0,0,0,0,0,0,0,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63342,US-7357652-B1,Leviton,1,0.986301,75,12,11,29,3,5,...,0,0,0,0,0,0,0,0,0,326
63343,US-8958680-B2,Leviton,1,0.986301,15,8,2,19,1,7,...,0,0,0,0,0,0,0,0,0,326
63346,US-9515437-B2,Leviton,1,0.986301,64,17,0,19,3,1,...,0,0,0,0,0,0,0,0,0,326
63347,US-9551454-B2,Leviton,1,0.986301,1,64,0,37,5,2,...,0,0,0,0,0,0,0,0,0,326


In [13]:
df['countries_in_family']

0                            ['AU' 'EP' 'CA' 'US']
1                                           ['US']
2                                           ['US']
3        ['JP' 'US' 'ES' 'HU' 'CN' 'EP' 'PL' 'DK']
4                            ['EP' 'CA' 'AU' 'US']
                           ...                    
63342                        ['US' 'CA' 'MX' 'CN']
63343                                  ['CA' 'US']
63346                                       ['US']
63347                                  ['US' 'CA']
63348                                  ['GB' 'US']
Name: countries_in_family, Length: 53616, dtype: object

In [18]:
# Count occurrences of 'US'

total_rows = df.shape[0]
print(total_rows)

us_occurrences = df['countries_in_family'].str.count('US').sum()
print(us_occurrences)

# Calculate percentage
us_percentage = (us_occurrences / total_rows) * 100

print(f"The percentage of 'US' in the 'countries_in_family' column is: {us_percentage:.2f}%")


53616
53616
The percentage of 'US' in the 'countries_in_family' column is: 100.00%


In [19]:
# Find the most frequent company in the dataset

total_rows = df.shape[0] 
most_frequent_company = df['company_name'].mode().iloc[0]
most_frequent_occurrences = df['company_name'].value_counts().max()  # Count occurrences of the most frequent company

# Calculate percentage
most_frequent_percentage = (most_frequent_occurrences / total_rows) * 100

print(f"The percentage of '{most_frequent_company}' in the 'company_name' column is: {most_frequent_percentage:.2f}%")


The percentage of 'Abbott' in the 'company_name' column is: 7.77%
